# Voter Turnout Dashboard

Interactive dashboard showing voter turnout data for 10 selected constituencies across three General Elections (2014, 2019, 2024).


## 1. Setup and Data Loading


In [1]:
import sys
import os
import importlib
sys.path.append('src')

import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.layouts import row, column, gridplot
from bokeh.models import Div

# Import modules
import data_loader
import data_processor
import create_dataset
import visualizations
import interactivity

# Reload modules to pick up any changes (useful during development)
importlib.reload(data_loader)
importlib.reload(data_processor)
importlib.reload(create_dataset)
importlib.reload(visualizations)
importlib.reload(interactivity)

# Import functions
from data_loader import load_all_data
from data_processor import create_standardized_dataset, aggregate_data, find_common_constituencies
from create_dataset import select_constituencies
from visualizations import (
    create_visualization_a,
    create_visualization_b,
    create_visualization_c,
    create_visualization_d,
    create_treemap
)
from interactivity import create_constituency_selector, create_year_filter, filter_data_by_selection

# Enable Bokeh in notebook
output_notebook()


Loading BokehJS ...

In [2]:
# Load or create dataset
dataset_path = 'data/voter_turnout_dataset.csv'

if os.path.exists(dataset_path):
    print("Loading existing dataset...")
    dataset = pd.read_csv(dataset_path)
else:
    print("Creating new dataset...")
    # Load all data
    data = load_all_data()

    # Find common constituencies
    common = find_common_constituencies(data, min_years=3)

    # Select 10 constituencies
    selected = select_constituencies(common, n=10)

    # Create standardized dataset
    dataset = create_standardized_dataset(data, selected)

    # Save dataset
    dataset.to_csv(dataset_path, index=False)
    dataset.to_excel('data/voter_turnout_dataset.xlsx', index=False)
    print("Dataset created and saved.")

print(f"\nDataset loaded: {len(dataset)} rows")
print(f"Constituencies: {dataset['PC_NAME'].nunique()}")
print(f"Years: {sorted(dataset['Year'].unique())}")


Loading existing dataset...

Dataset loaded: 30 rows
Constituencies: 10
Years: [np.int64(2014), np.int64(2019), np.int64(2024)]


## 2. Create Aggregated Data


In [3]:
# Aggregate data by year for visualizations A and B
aggregated = aggregate_data(dataset, group_by=['Year'])
print("Aggregated data by year:")
print(aggregated[['Year', 'Overall_Turnout', 'Male_Turnout', 'Female_Turnout']])


Aggregated data by year:
   Year  Overall_Turnout  Male_Turnout  Female_Turnout
0  2014            71.27         73.05           71.93
1  2019            72.68         73.63           73.38
2  2024            72.76         72.06           71.85


## 3. Create Visualizations


In [4]:
# Visualization A: Change in voter turnout over time
viz_a = create_visualization_a(aggregated)

# Visualization B: Change in voter turnout across genders
viz_b = create_visualization_b(aggregated)

# Visualization C: Distribution across constituencies and time
viz_c = create_visualization_c(dataset)

# Visualization D: Distribution across constituencies and genders
viz_d = create_visualization_d(dataset)

# Treemap: Constituencies sized by total votes
viz_treemap = create_treemap(dataset)

print("All visualizations created.")


AttributeError: unexpected attribute 'range_padding' to Range1d, possible attributes are bounds, end, js_event_callbacks, js_property_callbacks, max_interval, min_interval, name, reset_end, reset_start, start, subscribed_events, syncable or tags

## 4. Create Interactive Dashboard


In [ ]:
# Get unique constituencies and years
constituencies = sorted(dataset['PC_NAME'].unique().tolist())
years = sorted(dataset['Year'].unique().tolist())

# Pre-compute all aggregated data for all constituency combinations
# This allows CustomJS to filter client-side
print("Pre-computing aggregated data for all combinations...")

# Create a master data source with all constituency-year combinations
all_aggregated_data = {}
for const in constituencies:
    const_data = dataset[dataset['PC_NAME'] == const]
    for year in years:
        year_data = const_data[const_data['Year'] == year]
        if len(year_data) > 0:
            key = f"{const}_{year}"
            all_aggregated_data[key] = {
                'Constituency': const,
                'Year': year,
                'Overall_Turnout': year_data['Overall_Turnout'].iloc[0],
                'Male_Turnout': year_data['Male_Turnout'].iloc[0],
                'Female_Turnout': year_data['Female_Turnout'].iloc[0]
            }

# Store references to data sources from visualizations
viz_a_source = viz_a.renderers[0].data_source
viz_b_source = viz_b.renderers[0].data_source
viz_c_sources = [renderer.data_source for renderer in viz_c.renderers]
viz_d_source = viz_d.renderers[0].data_source
viz_treemap_source = viz_treemap.renderers[0].data_source

# Create widgets
constituency_selector = create_constituency_selector(constituencies, default_selection=constituencies)
year_filter = create_year_filter(years, default_selection=list(range(len(years))))

print(f"Constituencies: {len(constituencies)}")
print(f"Years: {years}")
print("Data sources and widgets ready")


In [ ]:
# Prepare full dataset for JavaScript filtering
# Create comprehensive data sources with all data points

# For Visualization A & B: Create data with all constituency-year combinations
full_data_list = []
for const in constituencies:
    const_data = dataset[dataset['PC_NAME'] == const]
    for year in years:
        year_data = const_data[const_data['Year'] == year]
        if len(year_data) > 0:
            full_data_list.append({
                'Constituency': const,
                'Year': year,
                'Year_str': str(year),
                'Overall_Turnout': float(year_data['Overall_Turnout'].iloc[0]),
                'Male_Turnout': float(year_data['Male_Turnout'].iloc[0]),
                'Female_Turnout': float(year_data['Female_Turnout'].iloc[0]),
                'Total_Votes': float(year_data['Total_Votes'].iloc[0])
            })

# Create master data source with all data
from bokeh.models import ColumnDataSource
master_source = ColumnDataSource(pd.DataFrame(full_data_list))

# For Visualization C: Pre-compute pivot data
pivot_full = dataset.pivot_table(
    values='Overall_Turnout',
    index='PC_NAME',
    columns='Year',
    aggfunc='mean'
).fillna(0)

# For Visualization D: Pre-compute constituency averages
const_avg_full = dataset.groupby('PC_NAME').agg({
    'Male_Turnout': 'mean',
    'Female_Turnout': 'mean'
}).reset_index()

print("Data prepared for JavaScript filtering")

# Create CustomJS callback for filtering (easiest approach for standalone output)
from bokeh.models import CustomJS

# Prepare data for JavaScript
constituencies_js = constituencies
years_js = [int(y) for y in years]

# Create JavaScript code to filter and aggregate data
filter_code = """
    // Get selected values
    const selected_constituencies = constituency_selector.value;
    const selected_year_indices = year_filter.active;
    const selected_years = selected_year_indices.map(i => years_js[i]);

    // Filter master data
    const master_data = master_source.data;
    const filtered_indices = [];

    for (let i = 0; i < master_data['Constituency'].length; i++) {
        const const_name = master_data['Constituency'][i];
        const year = master_data['Year'][i];

        if (selected_constituencies.includes(const_name) && selected_years.includes(year)) {
            filtered_indices.push(i);
        }
    }

    // Aggregate by year for Visualization A & B
    const year_data = {};
    for (const idx of filtered_indices) {
        const year = master_data['Year'][idx];
        const year_str = master_data['Year_str'][idx];

        if (!year_data[year]) {
            year_data[year] = {
                year_str: year_str,
                total_turnout: 0,
                total_male: 0,
                total_female: 0,
                count: 0
            };
        }
        year_data[year].total_turnout += master_data['Overall_Turnout'][idx];
        year_data[year].total_male += master_data['Male_Turnout'][idx];
        year_data[year].total_female += master_data['Female_Turnout'][idx];
        year_data[year].count += 1;
    }

    // Update Visualization A
    const years_sorted = Object.keys(year_data).sort();
    const viz_a_data = {
        'Year': years_sorted.map(y => year_data[y].year_str),
        'Turnout': years_sorted.map(y => year_data[y].total_turnout / year_data[y].count),
        'Year_num': years_sorted.map(y => parseInt(y))
    };
    viz_a_source.data = viz_a_data;
    viz_a_source.change.emit();

    // Update Visualization B
    const viz_b_data = {
        'Year': years_sorted.map(y => year_data[y].year_str),
        'Male': years_sorted.map(y => year_data[y].total_male / year_data[y].count),
        'Female': years_sorted.map(y => year_data[y].total_female / year_data[y].count),
        'Year_num': years_sorted.map(y => parseInt(y))
    };
    viz_b_source.data = viz_b_data;
    viz_b_source.change.emit();

    // Update Visualization C - filter by constituency and year
    const const_year_data = {};
    for (const idx of filtered_indices) {
        const const_name = master_data['Constituency'][idx];
        const year = master_data['Year'][idx];
        const key = const_name + '_' + year;
        if (!const_year_data[key]) {
            const_year_data[key] = {
                constituency: const_name,
                year: year,
                turnout: master_data['Overall_Turnout'][idx]
            };
        }
    }

    // Group by constituency and year
    const const_pivot = {};
    for (const key in const_year_data) {
        const item = const_year_data[key];
        if (!const_pivot[item.constituency]) {
            const_pivot[item.constituency] = {};
        }
        const_pivot[item.constituency][item.year] = item.turnout;
    }

    const filtered_constituencies = Object.keys(const_pivot).sort();

    // Update x_range
    if (viz_c.x_range.factors) {
        viz_c.x_range.factors = filtered_constituencies;
    }

    // Update each year's data source
    const year_to_idx = {2014: 0, 2019: 1, 2024: 2};
    for (const year of selected_years) {
        const idx = year_to_idx[year];
        if (idx !== undefined && idx < viz_c_sources.length) {
            const year_data_list = filtered_constituencies.map(c =>
                const_pivot[c] && const_pivot[c][year] ? const_pivot[c][year] : 0
            );
            viz_c_sources[idx].data = {
                'Constituency': filtered_constituencies,
                'Turnout': year_data_list
            };
            viz_c_sources[idx].change.emit();
        }
    }

    // Clear years not selected
    for (const year of [2014, 2019, 2024]) {
        if (!selected_years.includes(year)) {
            const idx = year_to_idx[year];
            if (idx !== undefined && idx < viz_c_sources.length) {
                viz_c_sources[idx].data = {'Constituency': [], 'Turnout': []};
                viz_c_sources[idx].change.emit();
            }
        }
    }

    // Update Visualization D - average by constituency
    const const_avg = {};
    for (const idx of filtered_indices) {
        const const_name = master_data['Constituency'][idx];
        if (!const_avg[const_name]) {
            const_avg[const_name] = {
                male: [],
                female: []
            };
        }
        const_avg[const_name].male.push(master_data['Male_Turnout'][idx]);
        const_avg[const_name].female.push(master_data['Female_Turnout'][idx]);
    }

    const filtered_const_d = Object.keys(const_avg).sort();
    const avg_male = filtered_const_d.map(c => {
        const vals = const_avg[c].male;
        return vals.reduce((a, b) => a + b, 0) / vals.length;
    });
    const avg_female = filtered_const_d.map(c => {
        const vals = const_avg[c].female;
        return vals.reduce((a, b) => a + b, 0) / vals.length;
    });

    if (viz_d.x_range.factors) {
        viz_d.x_range.factors = filtered_const_d;
    }

    viz_d_source.data = {
        'Constituency': filtered_const_d,
        'Male': avg_male,
        'Female': avg_female
    };
    viz_d_source.change.emit();
"""

# Create callback with all necessary references
callback = CustomJS(
    args={
        'constituency_selector': constituency_selector,
        'year_filter': year_filter,
        'master_source': master_source,
        'viz_a_source': viz_a_source,
        'viz_b_source': viz_b_source,
        'viz_c_sources': viz_c_sources,
        'viz_d_source': viz_d_source,
        'viz_treemap_source': viz_treemap_source,
        'viz_c': viz_c,
        'viz_d': viz_d,
        'years_js': years_js
    },
    code=filter_code
)

# Attach callback to widgets
constituency_selector.js_on_change('value', callback)
year_filter.js_on_change('active', callback)

# Create dashboard title
title = Div(text="""
<h1 style='text-align: center; margin-bottom: 20px;'>Voter Turnout Dashboard</h1>
<p style='text-align: center; color: #666;'>Analysis of 10 Selected Constituencies Across Three General Elections (2014, 2019, 2024)</p>
""", width=1200, height=80)

# Arrange visualizations in grid
top_row = row(viz_treemap, viz_a, viz_b)
bottom_row = row(viz_c, viz_d)

# Create sidebar with controls
sidebar = column(
    Div(text="<h3>Filters</h3>", width=200),
    constituency_selector,
    Div(text="<br>", height=20),
    Div(text="<b>Select Years:</b>", width=200),
    year_filter
)

# Combine everything
dashboard = column(
    title,
    row(sidebar, column(top_row, bottom_row))
)

# Display dashboard
show(dashboard)


## 5. Dataset Summary


In [ ]:
print("Selected Constituencies:")
for idx, (state, name) in enumerate(zip(dataset.groupby(['State', 'PC_NAME']).size().reset_index()['State'],
                                         dataset.groupby(['State', 'PC_NAME']).size().reset_index()['PC_NAME']), 1):
    print(f"{idx}. {name}, {state}")

print("\nDataset Summary by Year:")
summary = dataset.groupby('Year').agg({
    'Total_Electors': 'sum',
    'Total_Votes': 'sum',
    'Overall_Turnout': 'mean'
}).round(2)
print(summary)
